<a href="https://colab.research.google.com/github/straightjacket/ActiveMeasure/blob/master/Copy_of_how_to_use_lstm_for_one_to_many_many_to_one_and_many_to_many_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code in this colab notebook is heavily referenced from this [blog post by Usman Malik](https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras-part-2/).

# Imports and Setups

In [ ]:
!pip install -qq wandb

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import numpy as np

Create a Weigts and Biases account if you don't already have one: https://wandb.ai/signup

Copy and paste the API token in the input box below: https://wandb.ai/authorize

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# One to Many Sequence

## Data

In [ ]:
X, Y = [], []
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = []
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43]
[[2, 3], [5, 6], [8, 9], [11, 12], [14, 15], [17, 18], [20, 21], [23, 24], [26, 27], [29, 30], [32, 33], [35, 36], [38, 39], [41, 42], [44, 45]]


Let's reshape the input sequence to be of the shape `[batch, timesteps, feature]`.

In [ ]:
X = np.array(X).reshape(15, 1, 1)
Y = np.array(Y)

print(f"Shape of X: {X.shape} and shape of Y: {Y.shape}")

Shape of X: (15, 1, 1) and shape of Y: (15, 2)


## Model

In [ ]:
def get_model():
    inputs = layers.Input(shape=(1,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(2)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 2)                 102       
                                                                 
Total params: 10502 (41.02 KB)
Trainable params: 10502 (41.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train with W&B

In [ ]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

wandb: Currently logged in as: tsblount. Use `wandb login --relogin` to force relogin


Epoch 1/1000
4/4 [==============================] - 6s 89ms/step - loss: 460.4912 - val_loss: 1705.1488
Epoch 2/1000
4/4 [==============================] - 0s 12ms/step - loss: 457.2555 - val_loss: 1692.1852
Epoch 3/1000
4/4 [==============================] - 0s 12ms/step - loss: 453.3558 - val_loss: 1678.6246
Epoch 4/1000
4/4 [==============================] - 0s 12ms/step - loss: 449.7034 - val_loss: 1663.8917
Epoch 5/1000
4/4 [==============================] - 0s 12ms/step - loss: 445.6693 - val_loss: 1647.7982
Epoch 6/1000
4/4 [==============================] - 0s 11ms/step - loss: 441.1136 - val_loss: 1630.4921
Epoch 7/1000
4/4 [==============================] - 0s 13ms/step - loss: 435.5354 - val_loss: 1612.4459
Epoch 8/1000
4/4 [==============================] - 0s 15ms/step - loss: 430.5819 - val_loss: 1592.1207
Epoch 9/1000
4/4 [==============================] - 0s 12ms/step - loss: 425.9251 - val_loss: 1569.6791
Epoch 10/1000
4/4 [==============================] - 0s 12ms/ste

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,3998
batch/learning_rate,0.001
batch/loss,0.00473
epoch/epoch,999


## Prediction

In [ ]:
test_input = np.array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input)
print(test_output)

# Many to One Sequence

## Data

In [ ]:
X = np.array([x+1 for x in range(45)])
X = X.reshape(15,3,1)

Y = []
for x in X:
    Y.append(x.sum())
Y = np.array(Y)

print(X)
print(Y)

[[[ 1]
  [ 2]
  [ 3]]

 [[ 4]
  [ 5]
  [ 6]]

 [[ 7]
  [ 8]
  [ 9]]

 [[10]
  [11]
  [12]]

 [[13]
  [14]
  [15]]

 [[16]
  [17]
  [18]]

 [[19]
  [20]
  [21]]

 [[22]
  [23]
  [24]]

 [[25]
  [26]
  [27]]

 [[28]
  [29]
  [30]]

 [[31]
  [32]
  [33]]

 [[34]
  [35]
  [36]]

 [[37]
  [38]
  [39]]

 [[40]
  [41]
  [42]]

 [[43]
  [44]
  [45]]]
[  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132]


## Model

In [ ]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(1)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10451 (40.82 KB)
Trainable params: 10451 (40.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train

In [ ]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 4185.5581 - val_loss: 15951.5654
Epoch 2/1000
1/1 [==============================] - 0s 44ms/step - loss: 4168.1289 - val_loss: 15899.1279
Epoch 3/1000
1/1 [==============================] - 0s 44ms/step - loss: 4150.4614 - val_loss: 15846.1045
Epoch 4/1000
1/1 [==============================] - 0s 42ms/step - loss: 4132.5049 - val_loss: 15792.2656
Epoch 5/1000
1/1 [==============================] - 0s 40ms/step - loss: 4114.2188 - val_loss: 15737.1729
Epoch 6/1000
1/1 [==============================] - 0s 41ms/step - loss: 4095.6042 - val_loss: 15680.4795
Epoch 7/1000
1/1 [==============================] - 0s 40ms/step - loss: 4076.6104 - val_loss: 15622.4453
Epoch 8/1000
1/1 [==============================] - 0s 44ms/step - loss: 4057.1836 - val_loss: 15562.5938
Epoch 9/1000
1/1 [==============================] - 0s 39ms/step - loss: 4037.1746 - val_loss: 15500.8740
Epoch 10/1000
1/1 [=============================

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,1998
batch/learning_rate,0.001
batch/loss,0.0014
epoch/epoch,999


## Prediction

In [ ]:
test_input = np.array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[151.27563]]


# Many to Many Sequence

## Data

In [ ]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

print(X)
print(Y)

[[[  5]
  [ 10]
  [ 15]]

 [[ 20]
  [ 25]
  [ 30]]

 [[ 35]
  [ 40]
  [ 45]]

 [[ 50]
  [ 55]
  [ 60]]

 [[ 65]
  [ 70]
  [ 75]]

 [[ 80]
  [ 85]
  [ 90]]

 [[ 95]
  [100]
  [105]]

 [[110]
  [115]
  [120]]

 [[125]
  [130]
  [135]]

 [[140]
  [145]
  [150]]

 [[155]
  [160]
  [165]]

 [[170]
  [175]
  [180]]

 [[185]
  [190]
  [195]]

 [[200]
  [205]
  [210]]

 [[215]
  [220]
  [225]]

 [[230]
  [235]
  [240]]

 [[245]
  [250]
  [255]]

 [[260]
  [265]
  [270]]

 [[275]
  [280]
  [285]]

 [[290]
  [295]
  [300]]]
[[[ 20]
  [ 25]
  [ 30]]

 [[ 35]
  [ 40]
  [ 45]]

 [[ 50]
  [ 55]
  [ 60]]

 [[ 65]
  [ 70]
  [ 75]]

 [[ 80]
  [ 85]
  [ 90]]

 [[ 95]
  [100]
  [105]]

 [[110]
  [115]
  [120]]

 [[125]
  [130]
  [135]]

 [[140]
  [145]
  [150]]

 [[155]
  [160]
  [165]]

 [[170]
  [175]
  [180]]

 [[185]
  [190]
  [195]]

 [[200]
  [205]
  [210]]

 [[215]
  [220]
  [225]]

 [[230]
  [235]
  [240]]

 [[245]
  [250]
  [255]]

 [[260]
  [265]
  [270]]

 [[275]
  [280]
  [285]]

 [[290]
  [2

## Model

In [ ]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    encoder = layers.LSTM(100, activation="relu", return_sequences=False)(inputs)
    repeat = layers.RepeatVector(3)(encoder)
    decoder = layers.LSTM(100, activation='relu', return_sequences=True)(repeat)
    outputs = layers.TimeDistributed(layers.Dense(1))(decoder)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 repeat_vector (RepeatVecto  (None, 3, 100)            0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 3, 100)            80400     
                                                                 
 time_distributed (TimeDist  (None, 3, 1)              101       
 ributed)                                                        
                                                                 
Total params: 121301 (473.83 KB)
Trainable params: 121301 (47

## Train

In [ ]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

Epoch 1/1000
6/6 [==============================] - 3s 90ms/step - loss: 23010.3594 - val_loss: 68118.4141
Epoch 2/1000
6/6 [==============================] - 0s 24ms/step - loss: 17908.9414 - val_loss: 39609.1641
Epoch 3/1000
6/6 [==============================] - 0s 23ms/step - loss: 8017.0898 - val_loss: 18999.6914
Epoch 4/1000
6/6 [==============================] - 0s 22ms/step - loss: 4937.0698 - val_loss: 14773.3984
Epoch 5/1000
6/6 [==============================] - 0s 22ms/step - loss: 3630.5469 - val_loss: 10508.9697
Epoch 6/1000
6/6 [==============================] - 0s 25ms/step - loss: 2760.4155 - val_loss: 7666.9873
Epoch 7/1000
6/6 [==============================] - 0s 26ms/step - loss: 1838.2015 - val_loss: 4928.3018
Epoch 8/1000
6/6 [==============================] - 0s 22ms/step - loss: 1144.5371 - val_loss: 2821.6731
Epoch 9/1000
6/6 [==============================] - 0s 24ms/step - loss: 529.4293 - val_loss: 711.7253
Epoch 10/1000
6/6 [==============================]

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,5998
batch/learning_rate,0.001
batch/loss,1.67974
epoch/epoch,999


## Prediction

In [ ]:
test_input = np.array([300, 305, 310])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[[313.62485]
  [319.59918]
  [326.02444]]]
